#  4章　データ分析実践編
タイタニック・データセットを用いた分析


## 共通処理

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib | tail -n 1

In [ ]:
# ライブラリのimport

# NumPy用ライブラリ
import numpy as np

# Matplotlib中のpyplotライブラリのインポート
import matplotlib.pyplot as plt

# matplotlib日本語化対応ライブラリのインポート
import japanize_matplotlib

# pandas用ライブラリ
import pandas as pd

# データフレーム表示用関数
from IPython.display import display

# seaborn
import seaborn as sns

In [ ]:
# 表示オプション調整

# NumPy表示形式の設定
np.set_printoptions(
    suppress=True, precision=4, floatmode='fixed'
)

# グラフのデフォルトフォント指定
plt.rcParams["font.size"] = 14

# サイズ設定
plt.rcParams['figure.figsize'] = (6, 6)

# 方眼表示ON
plt.rcParams['axes.grid'] = True

# データフレームでの表示精度
pd.options.display.float_format = '{:.4f}'.format

# データフレームですべての項目を表示
pd.set_option("display.max_columns",None)

##  4.2 データ読み込み

### ファイルダウンロード

In [ ]:
url = 'https://raw.githubusercontent.com/makaishi2/samples/main/data/titanic-v2.csv'
!wget $url

### ファイル内容確認

In [ ]:
# ファイル名の定義
csv_fn = 'titanic-v2.csv'

# 先頭を確認
!head -3 $csv_fn

### データ読み込み　その1

In [ ]:
# データ読み込み　その1
df = pd.read_csv(
    csv_fn,
    na_values='?',
    quotechar="'")

### 問題判別

In [ ]:
# 問題の起きた行を確認
!head -130 $csv_fn | tail -1

### データ読み込み　その２

In [ ]:
# データ読み込み　その2
df = pd.read_csv(
    csv_fn, 
    na_values = '?',
    quotechar = "'", 
    escapechar = '\\')

# 結果確認
display(df.head(1))

## 4.3 データ確認・加工

### データ型確認


In [ ]:
# データ型確認(3.3.4項)
df.dtypes

In [ ]:
# 項目bodyの一部を表示
df[['body']].head()

### 3度目のデータ読み込み

In [ ]:
# 3度目のデータ読み込み
df = pd.read_csv(
    csv_fn, 
    na_values = '?',
    quotechar = "'", 
    escapechar = '\\',
    # 項目bodyを文字列型として読み込むよう指定
    dtype = {'body': object})

# 結果確認(データ型)
print(df.dtypes)

In [ ]:
# 項目bodyの先頭表示
display(df[['body']].head())

### 項目名変更

In [ ]:
 # 項目名変更　(3.3.2項)

columns = [
    '客室クラス', '生存状況', '氏名', '性別',
    '年齢', '兄弟_配偶者数', '親_子供数',
    '乗船券番号', '運賃', '客室番号', '乗船港',
    '救命ボート番号', '遺体識別番号', '自宅または目的地'
]
df.columns = columns

# 結果確認
display(df.head(2))

### 欠損値確認

In [ ]:
# 欠損値の確認　(3.3.3項)
print(df.isnull().sum())

### 統計量計算

In [ ]:
# 数値データの統計量計算　(3.3.5項)
df.describe()

In [ ]:
# 文字列型データの統計量確認　(3.3.5項)
df.describe(include=['O'])

### 値の出現回数確認

In [ ]:
# 出現回数をカウントしたい項目を抽出
df2 = df[['客室クラス', '生存状況', '性別', '乗船港']]

# 値の出現回数(3.3.6項)
for c in list(df2.columns):
    print(c)
    print(df[c].value_counts())
    print()

## 4.4 データ集計

### グループ毎の集計

In [ ]:
# 客室クラス毎の集計(3.4.5項)
df.groupby('客室クラス').mean()

### 出現頻度のクロス集計

In [ ]:
# 「客室クラス」「乗船港」を軸とした出現頻度分析
df_crosstab = pd.crosstab(
    index=df['客室クラス'], 
    columns=df['乗船港'],
    margins=True)

# 結果確認
display(df_crosstab)

### 項目値のクロス集計

In [ ]:
# 「客室クラス」と「性別」を軸とした、「生存状況」のクロス集計
df_pivot = df.pivot_table(
    # 「性別」「客室クラス」の２軸で分析
    index='性別', columns='客室クラス',
    # 分析対象項目は「生存状況」　集約関数は「平均」
    values='生存状況', aggfunc='mean')

# 結果確認
display(df_pivot)

## 4.5  データ可視化

#### 数値項目のヒストグラム表示

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)

# データフレームの数値項目でヒストグラム表示　(3.5.2項)
df.hist(bins=20, layout=(2, 3))
plt.tight_layout()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (4, 4)

# 運賃の詳細分布
df['運賃'].hist(bins=60)
plt.xlim(0, 150)
plt.title('運賃詳細分布')
plt.show()

#### 非数値項目の度数分布

In [ ]:
plt.rcParams['figure.figsize'] = (8, 4)

# 非数値項目の度数分布 (3.5.3項)
df2 = df[['性別', '乗船港']]

for i, c in enumerate(df2.columns):
    ax = plt.subplot(1, 2, i+1)
    df2[c].value_counts().plot(
        kind='bar', title=c, ax=ax)    

# レイアウトの調整    
plt.tight_layout()
plt.show()

### 箱ひげ図

In [ ]:
plt.rcParams['figure.figsize'] = (6, 6)

# 箱ひげ図の描画 (3.5.4項)
sns.boxplot(
    x='客室クラス', y='運賃', data=df, 
    palette=['blue', 'cyan', 'grey'])
plt.title('客室クラスと運賃の関係')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (6, 6)

# 箱ひげ図の描画 (3.5.4項)
sns.boxplot(
    x='客室クラス', y='運賃', data=df,
        palette=['blue', 'cyan', 'grey'])
plt.title('客室クラスと運賃の関係')

# y軸の上限を120に変更する
plt.ylim(0, 120)
plt.show()

### ヒートマップ

In [ ]:
# 「客室クラス」「性別」を軸とした「生存状況」クロス集計の可視化
df_pivot = df.pivot_table(
    # 「性別」「客室クラス」の２軸で分析
    index='性別', columns='客室クラス',
    # 分析対象項目は「生存状況」　集約関数は「平均」
    values='生存状況', aggfunc='mean')

# ヒートマップ表示
plt.rcParams['figure.figsize'] = (4, 4)
sns.heatmap(
    df_pivot, square=True, annot=True,
    fmt='.03f', cmap='Blues', cbar=False)
xlabel = df_pivot.columns.name
ylabel = df_pivot.index.name
plt.title(xlabel + ' vs '+ylabel)
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.show()

## 4.6 仮説と検証


### 生存状況と性別の関係

In [ ]:
plt.rcParams['figure.figsize'] = (4, 4)

# 生存状況と性別の関係
df.groupby('性別')['生存状況'].mean().plot(kind='bar')
plt.show()

### 生存状況と年齢の関係

In [ ]:
plt.rcParams['figure.figsize'] = (8, 4)

# 生存状況と年齢の関係
sns.histplot(
    data=df,  x='年齢', hue='生存状況', 
    palette=['blue', 'cyan'], multiple='dodge', 
    shrink=0.7)
plt.show()

### 生存状況と客室クラスの関係

In [ ]:
plt.rcParams['figure.figsize'] = (4, 4)

# 生存状況と客室クラスの関係
df.groupby('客室クラス')['生存状況'].mean().plot(kind='bar')
plt.show()

### 生存状況と乗船港の関係

In [ ]:
plt.rcParams['figure.figsize'] = (4, 4)

# 生存状況と乗船港の関係
df.groupby('乗船港')['生存状況'].mean().plot(kind='bar')
plt.show()


### 乗船港で生存状況が異なる理由



In [ ]:
# 「客室クラス」「乗船港」を軸とした出現頻度分析
# 列を軸に正規化する
df_crosstab = pd.crosstab(
    index=df['客室クラス'],
    columns=df['乗船港'],
    normalize='columns')

# ヒートマップ表示
plt.rcParams['figure.figsize'] = (4, 4)
sns.heatmap(
    df_crosstab, square=True, annot=True,
    fmt='.03f', cmap='Blues', cbar=False)
plt.show()

## 4.7 深掘り調査

### 項目「救命ボート」の追加

In [ ]:
# 項目「救命ボード」を追加
df['救命ボート'] = df['救命ボート番号'].notnull()

# 結果確認
display(df[['救命ボート番号', '救命ボート']].head(3))

### 「救命ボード」「生存状況」を軸とした出現頻度分析

In [ ]:
# 「救命ボード」「生存状況」を軸とした出現頻度分析
df_crosstab = pd.crosstab(
    index=df['生存状況'], 
    columns=df['救命ボート'])

# 結果確認
display(df_crosstab)

### 救命ボードなしで助かった人

In [ ]:
# 救命ボードなしで助かった人　(23名)
# queryメソッドを用いた深掘り調査(3.6.5項)
x1 = df.query(
    ' 生存状況 == 1 and 救命ボート == False ')

# 結果の一部確認
display(x1[[
    '客室クラス', '生存状況', '氏名', '性別', '年齢', 
    '救命ボート番号']].head(3))

In [ ]:
# 救命ボートなしで助かった人の性別分布
print(x1['性別'].value_counts())
print()

# 救命ボートなしで助かった人の割合を性別に集計
x11 = df.query('救命ボート == False')
print(x11.groupby('性別')['生存状況'].mean())

### 救命ボードに乗ったのに助からなかった人

In [ ]:
# 救命ボードに乗ったのに助からなかった人　(9名)
# queryメソッドを用いた深掘り調査(3.6.5項)
x2 = df.query(
    ' 生存状況 == 0 and 救命ボート == True ')

# 結果の一部確認
display(x2[[
    '客室クラス', '生存状況', '氏名', '性別', '年齢', 
    '救命ボート番号']].head(3))

In [ ]:
# 救命ボードに乗ったのに助からなかった人の性別分布
print(x2['性別'].value_counts())
print()

# 救命ボートに乗ったのに助からなかった人の割合を性別に集計
x22 = df.query('救命ボート == True')
print(1 - x22.groupby('性別')['生存状況'].mean())